# Deep Markov Model

In [5]:
from tqdm import tqdm

import torch
from torch import optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms, datasets
from tensorboardX import SummaryWriter
import numpy as np

In [6]:
batch_size = 128#128
epochs = 100
seed = 1
torch.manual_seed(seed)

In [7]:
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

In [8]:
#計測モデルとか
def get_ot(st,lmap,max_range):
    dis = torch.sqrt((st[:,0]-lmap[0])**2+(st[:,1]-lmap[1])**2)
    angle = torch.atan2((lmap[1]-st[:,1]),(lmap[0]-st[:,0]))-st[:,2]
    return torch.stack([dis,angle],1)
    
def get_all_ot(st,lmap,max_range):
    measure = get_ot(st,lmap[0],max_range)
    for l in range(1,len(lmap)):
        measure = torch.cat([measure, get_ot(st,lmap[l],max_range)],1)
    return torch.tensor(measure)

In [9]:
landmark_num = 10
start_pos = [2.0,4.0,0.0]#x0,y0,yaw0

In [10]:
landmark_dim = 2
x_dim = landmark_num*2
h_dim = 32 #32
hidden_dim = 32 #32
z_dim = 3
u_dim = 2
t_max = 66

In [11]:
def normalize(data, *args):
    if len(args) == 0:
        max_data = torch.max(abs(data))
        data=data/max_data
    else :
        max_data=torch.max(abs(torch.tensor(args)))
        data=data/max_data
    return data

def ot_normal(ot,range_ot):
    ot=ot.view(len(ot),10,2)
    ot[:,:,0]=normalize(ot[:,:,0],range_ot[0][0],range_ot[0][1])
    ot[:,:,1]=normalize(ot[:,:,1],range_ot[1][0],range_ot[1][1])
    ot = ot.view(len(ot),20)
    return ot

# def inv_ot_normal(ot,range_ot):
#     ot=ot.view(len(ot),10,2)
#     ot[:,:,0]=ot[:,:,0]*torch.max(range_ot[0][0],range_ot[0][1])
#     ot[:,:,1]=ot[:,:,1]*torch.max(range_ot[1][0],range_ot[1][1])
#     ot = ot.view(len(ot),20)
#     return ot

In [12]:
#データの読み込み
transform = transforms.Compose([transforms.ToTensor()])
kwargs = {'batch_size': batch_size, 'num_workers': 1, 'pin_memory': True}

range_ot=torch.tensor([[0.0,150.0],[-np.pi*2,np.pi*2]])#otの[v,r]の最大と最小

#data loader #とりあえず1時系列分を分身させて食わせてる
#[time,s_x,s_y,s_yaw,uv,ur,ot[1],,,,ot[N]]
data = np.loadtxt('vehicle_motion_data.csv', delimiter=',')
data = torch.tensor([data],dtype=torch.float32)
st = data[0,:,1:(1+z_dim)]
ut = data[0,:,(1+z_dim):(1+z_dim+u_dim)]
ot = data[0,:,(1+z_dim+u_dim):(1+z_dim+u_dim+x_dim)]
t_max=len(ot)
ot=ot_normal(ot,range_ot)

print(ot.size())
st=st.repeat(1000,1,1)
ut=ut.repeat(1000,1,1)
ot=ot.repeat(1000,1,1)
print(ot.size())


landmark = np.loadtxt('landmark_data.csv',delimiter=',')

train = torch.utils.data.TensorDataset(ot)
train_loader = torch.utils.data.DataLoader(train, shuffle=False,**kwargs)
test = torch.utils.data.TensorDataset(ot)
test_loader = torch.utils.data.DataLoader(test, shuffle=False,**kwargs)

torch.Size([66, 20])
torch.Size([1000, 66, 20])


In [13]:
from pixyz.models import Model
from pixyz.losses import KullbackLeibler, CrossEntropy, IterativeLoss, StochasticReconstructionLoss
from pixyz.distributions import Bernoulli, Normal, Deterministic
from pixyz.utils import print_latex

In [14]:
# from pixyz.distributions import Distribution
# from pixyz.utils import get_dict_values
# class Deterministic(Distribution):

#     def __init__(self, **kwargs):
#         super().__init__(**kwargs)

#     @property
#     def distribution_name(self):
#         return "Deterministic"

#     def sample(self, x_dict={}, return_all=True, **kwargs):
# #         print(x_dict["x"].size())
#         x_dict = self._check_input(x_dict)
#         _x_dict = get_dict_values(x_dict, self.input_var, return_dict=True)
#         output_dict = self.forward(**_x_dict)

#         if set(output_dict.keys()) != set(self._var):
#             raise ValueError("Output variables are not same as `var`.")

#         if return_all:
#             x_dict.update(output_dict)
#             return x_dict

#         return output_dict

#     def sample_mean(self, x_dict):
#         return self.sample(x_dict, return_all=False)[self._var[0]]

In [15]:
class RNN(Deterministic):
    def __init__(self):
        super(RNN, self).__init__(cond_var=["x"], var=["h"],name="q")
        self.rnn = nn.GRU(x_dim, h_dim, bidirectional=True)
#         self.h0 = torch.zeros(2, batch_size, self.rnn.hidden_size).to(device)
        self.h0 = nn.Parameter(torch.zeros(2, 1, self.rnn.hidden_size))
        self.hidden_size = self.rnn.hidden_size
        
    def forward(self, x):
#         if(x.size(1)!=128):
#             x=x.repeat(66,1,1)
#         print("xsize",str(x.size()))
        h0 = self.h0.expand(2, x.size(1), self.rnn.hidden_size).contiguous()
#         print(self.rnn(x,h0))
        h, _ = self.rnn(x, h0)
#         print("hsize",str(h.size()))
        return {"h": h}

In [16]:
# class Phi_x(nn.Module):
#     def __init__(self):
#         super(Phi_x, self).__init__()
#         self.fc0 = nn.Linear(x_dim, h_dim)
#     def forward(self, x):
#         return F.relu(self.fc0(x))

# f_phi_x = Phi_x().to(device)

class RNN(Deterministic):
     def __init__(self):
         super(RNN, self).__init__(cond_var=["x","h_prev"], var=["h"],name="q")
         self.gru1 = nn.GRUCell(h_dim, h_dim).to(device)
         self.hidden_size = h_dim
         self.f_phi_x = f_phi_x
        
        
     def forward(self, x, h_prev):
         print(x.size(),h_prev.size())
         h = self.gru1(self.f_phi_x(x), h_prev)
         return {"h": h}

#class RNN(Deterministic):
#     def __init__(self):
#         super(RNN, self).__init__(cond_var=["x","h_prev"], var=["h"],name="q")
#         self.gru1 = nn.GRUCell(h_dim, h_dim).to(device)
#         self.hidden_size = h_dim
#         self.f_phi_x = f_phi_x
        
        
#     def forward(self, x, h_prev):
#         print(x.size(),h_prev.size())
#         h = self.gru1(self.f_phi_x(x), h_prev)
#         return {"h": h}

In [17]:
# class Generator(Bernoulli):
#     def __init__(self):
#         super(Generator, self).__init__(cond_var=["z"], var=["x"])
#         self.fc1 = nn.Linear(z_dim, hidden_dim)
#         self.fc2 = nn.Linear(hidden_dim, x_dim)
    
#     def forward(self, z):
# #         print(z.size()) #[128,3]
#         h = F.relu(self.fc1(z))
#         return {"probs": torch.sigmoid(self.fc2(h))}
class Generator(Normal):
    def __init__(self):
        super(Generator, self).__init__(cond_var=["z"], var=["x"])
    
    def forward(self, z):#計測モデルそのまま
#         print("z.size",str(z.size()))
        ot=get_all_ot(z,landmark,[1000,1000])
        ot=ot_normal(ot,range_ot) #データを正規化しなおしてる
        return {"loc": ot,"scale":torch.tensor(0.1).to(device)}

In [18]:
class Inference(Normal):
    def __init__(self):
        super(Inference, self).__init__(cond_var=["h", "z_prev"], var=["z"], name="q")
        self.fc1 = nn.Linear(z_dim, h_dim*2)
        self.fc21 = nn.Linear(h_dim*2, z_dim)
        self.fc22 = nn.Linear(h_dim*2, z_dim)
        
    def forward(self, h, z_prev):
#         print("h.size",str(h.size()))
        h_z = torch.tanh(self.fc1(z_prev))
        h = 0.5 * (h + h_z)
        return {"loc": self.fc21(h), "scale": F.softplus(self.fc22(h))}

In [19]:
class Prior(Normal):#通常ここには動作モデルをいれる
    def __init__(self):
        super(Prior, self).__init__(cond_var=["z_prev"], var=["z"],name="p")
        self.fc1 = nn.Linear(z_dim, hidden_dim)
        self.fc21 = nn.Linear(hidden_dim, z_dim)
        self.fc22 = nn.Linear(hidden_dim, z_dim)
        
    def forward(self, z_prev):
        h = F.relu(self.fc1(z_prev))
        return {"loc": self.fc21(h), "scale": F.softplus(self.fc22(h))}

In [20]:
prior = Prior().to(device)
encoder = Inference().to(device)
decoder = Generator().to(device)
rnn = RNN().to(device)

In [21]:
print(prior)
print("*"*80)
print(encoder)
print("*"*80)
print(decoder)
print("*"*80)
print(rnn)

Distribution:
  p(z|z_{prev})
Network architecture:
  Prior(
    name=p, distribution_name=Normal,
    var=['z'], cond_var=['z_prev'], input_var=['z_prev'], features_shape=torch.Size([])
    (fc1): Linear(in_features=3, out_features=32, bias=True)
    (fc21): Linear(in_features=32, out_features=3, bias=True)
    (fc22): Linear(in_features=32, out_features=3, bias=True)
  )
********************************************************************************
Distribution:
  q(z|h,z_{prev})
Network architecture:
  Inference(
    name=q, distribution_name=Normal,
    var=['z'], cond_var=['h', 'z_prev'], input_var=['h', 'z_prev'], features_shape=torch.Size([])
    (fc1): Linear(in_features=3, out_features=64, bias=True)
    (fc21): Linear(in_features=64, out_features=3, bias=True)
    (fc22): Linear(in_features=64, out_features=3, bias=True)
  )
********************************************************************************
Distribution:
  p(x|z)
Network architecture:
  Generator(
    name=p, 

In [22]:
generate_from_prior = prior * decoder
full_encoder = rnn*encoder
full_decoder = encoder*decoder
print(generate_from_prior)
print_latex(generate_from_prior)
print(full_encoder)
print_latex(full_encoder)

Distribution:
  p(x,z|z_{prev}) = p(x|z)p(z|z_{prev})
Network architecture:
  Prior(
    name=p, distribution_name=Normal,
    var=['z'], cond_var=['z_prev'], input_var=['z_prev'], features_shape=torch.Size([])
    (fc1): Linear(in_features=3, out_features=32, bias=True)
    (fc21): Linear(in_features=32, out_features=3, bias=True)
    (fc22): Linear(in_features=32, out_features=3, bias=True)
  )
  Generator(
    name=p, distribution_name=Normal,
    var=['x'], cond_var=['z'], input_var=['z'], features_shape=torch.Size([])
  )
Distribution:
  p(z,h|z_{prev},x) = q(z|h,z_{prev})q(h|x)
Network architecture:
  RNN(
    name=q, distribution_name=Deterministic,
    var=['h'], cond_var=['x'], input_var=['x'], features_shape=torch.Size([])
    (rnn): GRU(20, 32, bidirectional=True)
  )
  Inference(
    name=q, distribution_name=Normal,
    var=['z'], cond_var=['h', 'z_prev'], input_var=['h', 'z_prev'], features_shape=torch.Size([])
    (fc1): Linear(in_features=3, out_features=64, bias=True)


<IPython.core.display.Math object>

In [23]:
# from pixyz.losses.losses import SetLoss
# class StochasticReconstructionLoss(SetLoss):

#     def __init__(self, encoder, decoder, input_var=None):

#         if input_var is None:
#             input_var = encoder.input_var

#         if not(set(decoder.var) <= set(input_var)):
#             raise ValueError("Variable {} (in the `{}` class) is not included"
#                              " in `input_var` of the `{}` class.".format(decoder.var,
#                                                                          decoder.__class__.__name__,
#                                                                          encoder.__class__.__name__))

#         print(input_var)
#         loss = -decoder.log_prob().expectation(encoder,input_var)
#         super().__init__(loss)


In [24]:
# step_loss = CrossEntropy(encoder,decoder) + KullbackLeibler(encoder, prior)
step_loss = StochasticReconstructionLoss(full_encoder,decoder) + KullbackLeibler(encoder, prior)
# step_loss = step_loss.mean()
_loss = IterativeLoss(step_loss, max_iter=t_max, 
                      series_var=["x","h"], update_value={"z": "z_prev"})
# _loss = IterativeLoss(step_loss, max_iter=t_max, 
#                       series_var=["x"], update_value={"z": "z_prev","h":"h_prev"})
# loss=_loss
loss = _loss.expectation(rnn).mean()
print_latex(step_loss)

<IPython.core.display.Math object>

In [25]:
print(decoder.get_log_prob)

<bound method DistributionBase.get_log_prob of Generator(
  name=p, distribution_name=Normal,
  var=['x'], cond_var=['z'], input_var=['z'], features_shape=torch.Size([])
)>


In [26]:
print(full_encoder.input_var)

['z_prev', 'x']


In [27]:
print_latex(_loss)

<IPython.core.display.Math object>

In [28]:
print_latex(loss)

<IPython.core.display.Math object>

In [29]:
dmm = Model(loss, distributions=[rnn,encoder, decoder, prior], 
            optimizer=optim.RMSprop, optimizer_params={"lr": 5e-4}, clip_grad_value=10)

In [30]:
print(dmm)
print_latex(dmm)

Distributions (for training): 
  q(h|x), q(z|h,z_{prev}), p(x|z), p(z|z_{prev}) 
Loss function: 
  mean \left(\mathbb{E}_{q(h|x)} \left[\sum_{t=1}^{66} \left(D_{KL} \left[q(z|h,z_{prev})||p(z|z_{prev}) \right] - \mathbb{E}_{p(z,h|z_{prev},x)} \left[\log p(x|z) \right]\right) \right] \right) 
Optimizer: 
  RMSprop (
  Parameter Group 0
      alpha: 0.99
      centered: False
      eps: 1e-08
      lr: 0.0005
      momentum: 0
      weight_decay: 0
  )


<IPython.core.display.Math object>

In [31]:
def data_loop(epoch, loader, model, device, train_mode=False):
    mean_loss = 0
    for idx,[data] in enumerate(tqdm(loader)):#batch_idx, (data, _) in enumerate(tqdm(loader)):
        data = data.to(device)
        batch_size = data.size()[0]
        x = data.transpose(0, 1) #多分転置してるだけ
        z_prev = torch.tensor(start_pos)
        z_prev = z_prev.repeat(batch_size, 1).to(device)
#         h_prev = torch.zeros(batch_size, h_dim).to(device)
        print(z_prev.size())
        if train_mode:
            mean_loss += model.train({'x': x, 'z_prev': z_prev}).item() * batch_size
        else:
            mean_loss += model.test({'x': x, 'z_prev': z_prev}).item() * batch_size
    mean_loss /= len(loader.dataset)
    if train_mode:
        print('Epoch: {} Train loss: {:.4f}'.format(epoch, mean_loss))
    else:
        print('Test loss: {:.4f}'.format(mean_loss))
    return mean_loss

In [32]:
def plot_image_from_latent(batch_size):
    x = []
    z_prev = torch.zeros(batch_size, z_dim).to(device)
    for step in range(t_max):
        samples = generate_from_prior.sample({'z_prev': z_prev})
        x_t = decoder.sample_mean({"z": samples["z"]})
        z_prev = samples["z"]
        x.append(x_t[None, :])
    x = torch.cat(x, dim=0).transpose(0, 1)
    return x

In [33]:
writer = SummaryWriter()

history = {"train_loss":[],"test_loss":[]}

for epoch in range(1, epochs + 1):
    train_loss = data_loop(epoch, train_loader, dmm, device, train_mode=True)
    test_loss = data_loop(epoch, test_loader, dmm, device)

    writer.add_scalar('train_loss', train_loss, epoch)
    writer.add_scalar('test_loss', test_loss, epoch)
    
    history["train_loss"].append(train_loss)
    history["test_loss"].append(test_loss)

    sample = plot_image_from_latent(batch_size)[:, None][1,:]
    writer.add_image('Image_from_latent', sample, epoch)

  0%|          | 0/8 [00:00<?, ?it/s]

torch.Size([128, 3])


RuntimeError: input must have 3 dimensions, got 2

In [ ]:
import matplotlib.pyplot as plt

plt.ylabel('$loss$', fontsize=16)
plt.xlabel('$epoch$', fontsize=16)
ay=plt.gca()
plt.title("train_loss")
plt.plot(range(epochs), [i+0.5 for i in history["train_loss"]])
plt.show()
ay=plt.gca()
plt.title("test_loss")
plt.plot(range(epochs), [i+0.4 for i in history["test_loss"]])
plt.show()

In [ ]:

inference_net = rnn*encoder
test_o = data[0,:,(1+z_dim+u_dim):(1+z_dim+u_dim+x_dim)]
test_o = torch.tensor(test_o).reshape(1,len(test_o),20).to(device)
z_prev = torch.tensor(start_pos).to(device)
infered_result = inference_net.sample({"x":test_o,"z_prev":z_prev})["z"].to("cpu")
infered_result=infered_result.numpy()

plt.plot(infered_result[:,:, 0], infered_result[:,:, 1], "co")

for i in range(len(infered_result[0])-1):
    plt.plot([infered_result[0][i][0],infered_result[0][i+1][0]],[infered_result[0][i][1],infered_result[0][i+1][1]],"r")

plt.show()